Note: I left the code to generate the htfs for each test system in the notebook, but if you don't want to re-generate, you can use the pickled htf that I saved (I left the full path in for you). I generated these htfs based after you added support for `lambda_{i}_bonds_old/new` and `scale_lambda_0_bonds`

In [11]:
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from simtk import unit, openmm
from perses.tests.utils import compute_potential_components
from openmmtools.constants import kB
from perses.dispersed.utils import configure_platform
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
import numpy as np
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.testsystems import AlanineDipeptideVacuum, AlanineDipeptideExplicit
import itertools
from perses.tests.test_topology_proposal import generate_dipeptide_top_pos_sys, generate_atp
import pickle
import copy

#############################################
# CONSTANTS
#############################################
temperature = 298.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
REFERENCE_PLATFORM = openmm.Platform.getPlatformByName("CUDA")


conducting subsequent work with the following platform: CUDA


In [2]:
def test_torsion_energies(htf, is_old=True, is_solvated=False, check_scale=False):
    htf_copy = copy.deepcopy(htf)
    
    # Get periodic torsion force and old/new positions
    system = htf._topology_proposal.old_system if is_old else htf._topology_proposal.new_system
    periodic_torsion_force = system.getForce(2) 
    positions = htf.old_positions(htf.hybrid_positions) if is_old else htf.new_positions(htf.hybrid_positions)
    
    # Get custom torsion force and hybrid positions
    torsion_force_index = 3 if is_solvated else 2
    hybrid_system = htf.hybrid_system
    custom_torsion_force = hybrid_system.getForce(torsion_force_index)
    hybrid_positions = htf.hybrid_positions
    
    # Remove nonbonded/exception forces
    for i in range(hybrid_system.getNumForces(), hybrid_system.getNumForces() - 4, -1):
        hybrid_system.removeForce(i - 1)
    
    # Set global parameters
    lambda_old = 1 if is_old else 0
    lambda_new = 0 if is_old else 1
    for i in range(custom_torsion_force.getNumGlobalParameters()):
        if custom_torsion_force.getGlobalParameterName(i) == 'lambda_0_torsions_old':
            custom_torsion_force.setGlobalParameterDefaultValue(i, lambda_old)
        elif custom_torsion_force.getGlobalParameterName(i) == 'lambda_0_torsions_new':
            custom_torsion_force.setGlobalParameterDefaultValue(i, lambda_new)

    # Zero the unique old/new torsions in the custom torsion force
    hybrid_to_torsion_indices = htf._hybrid_to_new_torsion_indices if is_old else htf._hybrid_to_old_torsion_indices
    print(hybrid_to_torsion_indices)
    for hybrid_idx, idx in hybrid_to_torsion_indices.items():
        p1, p2, p3, p4, hybrid_params = custom_torsion_force.getTorsionParameters(hybrid_idx)
        hybrid_params = list(hybrid_params)
#         index_to_zero = -4 if is_old else -1
#         hybrid_params[index_to_zero] *= 0
        hybrid_params[-4] *= 0
        hybrid_params[-1] *= 0
        custom_torsion_force.setTorsionParameters(hybrid_idx, p1, p2, p3, p4, hybrid_params)
        
    ## Get energy components of standard torsion force
    platform = configure_platform(REFERENCE_PLATFORM)
    thermostate_other = ThermodynamicState(system=system, temperature=temperature)
    integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
    context_other = thermostate_other.create_context(integrator_other)
    context_other.setPositions(positions)
    components_other = compute_potential_components(context_other, beta=beta)
    print(components_other)
    
    ## Get energy components of custom torsion force
    if is_solvated:
        custom_torsion_force.setUsesPeriodicBoundaryConditions(True)
    thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
    integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
    context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
    context_hybrid.setPositions(hybrid_positions)
    components_hybrid = compute_potential_components(context_hybrid, beta=beta)
    print(components_hybrid)
    
    assert np.isclose([components_other[2][1]], [components_hybrid[2][1]])
    
    print("Success! Custom torsion force and standard torsion force energies are equal!")
    
    if check_scale:
        
        ## Get custom torsion force and hybrid positions
        torsion_force_index = 3 if is_solvated else 2
        hybrid_system = htf_copy.hybrid_system
        custom_torsion_force = hybrid_system.getForce(torsion_force_index)
        hybrid_positions = htf_copy.hybrid_positions
        
        # Remove nonbonded/exception forces
        for i in range(hybrid_system.getNumForces(), hybrid_system.getNumForces() - 4, -1):
            hybrid_system.removeForce(i - 1)
        
        if is_solvated:
            custom_torsion_force.setUsesPeriodicBoundaryConditions(True)
        
        ## Get energy components of custom torsion force
        thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
        integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
        context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
        context_hybrid.setPositions(hybrid_positions)
        components_hybrid = compute_potential_components(context_hybrid, beta=beta)
        print(components_hybrid)
        
        # Set `scale_lambda_{i}` to 0.5
        for i in range(custom_torsion_force.getNumGlobalParameters()):
            if custom_torsion_force.getGlobalParameterName(i) == 'scale_lambda_0_torsions':
                custom_torsion_force.setGlobalParameterDefaultValue(i, 0.5)
            elif custom_torsion_force.getGlobalParameterName(i) == 'interscale_lambda_0_torsions':
                custom_torsion_force.setGlobalParameterDefaultValue(i, 0.5)
        
        ## Get energy components of custom torsion force iwth scaling
        thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
        integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
        context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
        context_hybrid.setPositions(hybrid_positions)
        components_hybrid_scaled = compute_potential_components(context_hybrid, beta=beta)
        print(components_hybrid_scaled)
        
        assert not np.isclose([components_hybrid[2][1]], [components_hybrid_scaled[2][1]])
        
        print("Success! Scaling the torsion force changes the energy")

# Alanine dipeptide in vacuum

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [13]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [14]:
test_torsion_energies(htf)

{50: 8, 51: 9, 52: 10, 58: 16, 59: 17, 60: 18, 61: 19, 62: 20, 63: 21, 64: 22, 65: 23, 66: 24, 70: 28, 71: 29, 72: 30, 73: 31, 74: 32, 75: 33, 76: 34, 79: 37, 80: 38, 81: 39, 84: 42, 85: 43, 86: 44, 87: 45, 88: 46, 89: 47, 90: 48, 91: 49, 92: 50, 93: 51, 94: 52, 95: 53, 99: 57, 100: 58, 101: 59, 102: 60, 103: 61, 104: 62, 105: 63, 106: 64, 107: 65, 108: 66, 109: 67}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478365356520876), ('HarmonicAngleForce', 0.6112775462128573), ('PeriodicTorsionForce', 16.285398289983085), ('NonbondedForce', -39.449927745600164), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 1.296256769802195), ('CustomAngleForce', 3.8580778529179494), ('CustomTorsionForce', 16.285403285921724), ('AndersenThermostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [15]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [16]:
test_torsion_energies(htf, is_old=False)

{8: 8, 9: 9, 10: 10, 16: 16, 17: 17, 18: 18, 24: 24, 25: 25, 26: 26, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 1.2957330745733426), ('HarmonicAngleForce', 3.403930427792299), ('PeriodicTorsionForce', 30.268405909483896), ('NonbondedForce', 388.51299936325097), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 1.296256769802195), ('CustomAngleForce', 3.8580778529179494), ('CustomTorsionForce', 30.26841040928556), ('AndersenThermostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


### Test one alchemical region, one scale region

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [17]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [18]:
htf.hybrid_system.getForces()

[<simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2ac8cdffb8d0> >,
 <simtk.openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2ac8cdffb990> >,
 <simtk.openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2ac8cdffb750> >,
 <simtk.openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2ac8cdffb810> >,
 <simtk.openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2ac8cdffb600> >,
 <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2ac8cdffb690> >,
 <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2ac8cdffb5d0> >]

In [19]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[10, 11, 12, 13, 22, 23, 24, 25, 26, 27, 28, 29]

In [20]:
test_torsion_energies(htf)

{50: 8, 51: 9, 52: 10, 58: 16, 59: 17, 60: 18, 61: 19, 62: 20, 63: 21, 64: 22, 65: 23, 66: 24, 70: 28, 71: 29, 72: 30, 73: 31, 74: 32, 75: 33, 76: 34, 79: 37, 80: 38, 81: 39, 84: 42, 85: 43, 86: 44, 87: 45, 88: 46, 89: 47, 90: 48, 91: 49, 92: 50, 93: 51, 94: 52, 95: 53, 99: 57, 100: 58, 101: 59, 102: 60, 103: 61, 104: 62, 105: 63, 106: 64, 107: 65, 108: 66, 109: 67}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478365356521381), ('HarmonicAngleForce', 0.6112775462128995), ('PeriodicTorsionForce', 16.285398289983064), ('NonbondedForce', -39.4499277456003), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 3.0809325282327777), ('CustomAngleForce', 12.79191873055503), ('CustomTorsionForce', 16.285403285921724), ('AndersenThermostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [21]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [22]:
test_torsion_energies(htf, is_old=False)

{8: 8, 9: 9, 10: 10, 16: 16, 17: 17, 18: 18, 24: 24, 25: 25, 26: 26, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 3.0804088330039257), ('HarmonicAngleForce', 12.337771305429378), ('PeriodicTorsionForce', 28.091644511264406), ('NonbondedForce', 56.74765841438245), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 3.0809325282327777), ('CustomAngleForce', 12.79191873055503), ('CustomTorsionForce', 28.091648948068066), ('AndersenThermostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


# Alanine dipeptide in solvent

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [23]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [24]:
test_torsion_energies(htf, is_solvated=True)

{50: 8, 51: 9, 52: 10, 58: 16, 59: 17, 60: 18, 61: 19, 62: 20, 63: 21, 64: 22, 65: 23, 66: 24, 70: 28, 71: 29, 72: 30, 73: 31, 74: 32, 75: 33, 76: 34, 79: 37, 80: 38, 81: 39, 84: 42, 85: 43, 86: 44, 87: 45, 88: 46, 89: 47, 90: 48, 91: 49, 92: 50, 93: 51, 94: 52, 95: 53, 99: 57, 100: 58, 101: 59, 102: 60, 103: 61, 104: 62, 105: 63, 106: 64, 107: 65, 108: 66, 109: 67}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.034783653565214685), ('HarmonicAngleForce', 0.6112775462129015), ('PeriodicTorsionForce', 16.285398289983082), ('NonbondedForce', -1017.9419702202207), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 2.108758935146324), ('CustomAngleForce', 9.27903555849094), ('CustomTorsionForce', 16.285403285921724), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force a

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [25]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [26]:
test_torsion_energies(htf, is_old=False, is_solvated=True)

{8: 8, 9: 9, 10: 10, 16: 16, 17: 17, 18: 18, 24: 24, 25: 25, 26: 26, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 2.108235239917542), ('HarmonicAngleForce', 8.824888133365281), ('PeriodicTorsionForce', 25.495955289982533), ('NonbondedForce', -1054.9212696042337), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 2.108758935146324), ('CustomAngleForce', 9.27903555849094), ('CustomTorsionForce', 25.495959407354306), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


### Test one alchemical region, one scale region

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [27]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [28]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[10, 11, 12, 13, 1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564]

In [29]:
test_torsion_energies(htf, is_solvated=True)

{50: 8, 51: 9, 52: 10, 58: 16, 59: 17, 60: 18, 61: 19, 62: 20, 63: 21, 64: 22, 65: 23, 66: 24, 70: 28, 71: 29, 72: 30, 73: 31, 74: 32, 75: 33, 76: 34, 79: 37, 80: 38, 81: 39, 84: 42, 85: 43, 86: 44, 87: 45, 88: 46, 89: 47, 90: 48, 91: 49, 92: 50, 93: 51, 94: 52, 95: 53, 99: 57, 100: 58, 101: 59, 102: 60, 103: 61, 104: 62, 105: 63, 106: 64, 107: 65, 108: 66, 109: 67}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.034783653565214685), ('HarmonicAngleForce', 0.6112775462129015), ('PeriodicTorsionForce', 16.285398289983082), ('NonbondedForce', -995.1493033223735), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.915410547041641), ('CustomAngleForce', 9.661801162668427), ('CustomTorsionForce', 16.285403285921724), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force a

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [30]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [31]:
test_torsion_energies(htf, is_old=False, is_solvated=True, check_scale=True)

{8: 8, 9: 9, 10: 10, 16: 16, 17: 17, 18: 18, 24: 24, 25: 25, 26: 26, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.9148868518127816), ('HarmonicAngleForce', 9.207653737542962), ('PeriodicTorsionForce', 27.875973709878817), ('NonbondedForce', -940.6669304074791), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.915410547041641), ('CustomAngleForce', 9.661801162668427), ('CustomTorsionForce', 27.875978867637855), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.915410547041641), ('CustomAngleForce', 9.661801162668427), ('CustomTorsionForce', 16.285403285921724), ('AndersenThermostat',

# CDK2 transformation in solvent

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [32]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [33]:
test_torsion_energies(htf, is_solvated=True)

{235: 78, 270: 113, 297: 140, 298: 141, 303: 146}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.608082977954756), ('HarmonicAngleForce', 29.199523609209102), ('PeriodicTorsionForce', 15.076747992617829), ('NonbondedForce', 3350.162471334583), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 31.450793105256693), ('CustomTorsionForce', 15.076753795301011), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [34]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [35]:
test_torsion_energies(htf, is_old=False, is_solvated=True)

{70: 70, 112: 112, 142: 142, 143: 143, 148: 148}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.519508183899664), ('HarmonicAngleForce', 31.40442991986144), ('PeriodicTorsionForce', 15.078153431480851), ('NonbondedForce', 3363.615071727043), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 31.450793105256693), ('CustomTorsionForce', 15.078159774398037), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


### Test one alchemical region, one scale region

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [36]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent_scale_regions.pickle", "rb") as f:
    htf = pickle.load(f)

ModuleNotFoundError: No module named 'oechem'

In [ ]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

In [ ]:
test_torsion_energies(htf, is_solvated=True)

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [ ]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent_scale_regions.pickle", "rb") as f:
    htf = pickle.load(f)

In [ ]:
test_torsion_energies(htf, is_old=False, is_solvated=True, check_scale=True)

# 8mer in solvent

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [39]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [40]:
test_torsion_energies(htf, is_solvated=True)

{432: 39, 433: 40, 434: 41, 451: 58, 452: 59, 453: 60, 454: 61, 455: 62, 456: 63, 459: 66, 460: 67, 461: 68, 462: 69, 463: 70, 464: 71, 465: 72, 466: 73, 467: 74, 468: 75, 469: 76, 470: 77, 481: 88, 482: 89, 483: 90, 484: 91, 485: 92, 486: 93, 487: 94, 488: 95, 489: 96, 490: 97, 491: 98, 492: 99, 493: 100, 494: 101, 495: 102, 496: 103, 497: 104, 498: 105, 499: 106, 500: 107, 501: 108}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 7.349099420286866), ('HarmonicAngleForce', 12.529213143558692), ('PeriodicTorsionForce', 157.29212321868366), ('NonbondedForce', -10413.63480329325), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 8.88521073611831), ('CustomAngleForce', 382.4742987998808), ('CustomTorsionForce', 157.2921325778617), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom t

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [41]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent.pickle", "rb") as f:
    htf = pickle.load(f)


In [42]:
test_torsion_energies(htf, is_old=False, is_solvated=True)

{39: 39, 40: 40, 41: 41, 58: 58, 59: 59, 60: 60, 63: 63, 64: 64, 65: 65, 76: 76, 77: 77, 78: 78, 79: 79, 80: 80, 81: 81}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 8.839279898124524), ('HarmonicAngleForce', 382.3538632504383), ('PeriodicTorsionForce', 168.47244240570083), ('NonbondedForce', -6861.17431642076), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 8.88521073611831), ('CustomAngleForce', 382.4742987998808), ('CustomTorsionForce', 168.4724512217227), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


### Test one alchemical region, one scale region

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [43]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [44]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[24, 25, 26, 21, 5575, 5576, 5577, 5578, 5579]

In [45]:
test_torsion_energies(htf, is_solvated=True)

{432: 39, 433: 40, 434: 41, 451: 58, 452: 59, 453: 60, 454: 61, 455: 62, 456: 63, 459: 66, 460: 67, 461: 68, 462: 69, 463: 70, 464: 71, 465: 72, 466: 73, 467: 74, 468: 75, 479: 86, 480: 87, 481: 88, 482: 89, 483: 90, 484: 91, 485: 92, 486: 93}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 7.349099420288431), ('HarmonicAngleForce', 12.529213143555554), ('PeriodicTorsionForce', 157.292123218685), ('NonbondedForce', -10511.125832891092), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 12.665182674760329), ('CustomAngleForce', 222.15532081099008), ('CustomTorsionForce', 157.2921325778617), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [46]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [47]:
test_torsion_energies(htf, is_old=False, is_solvated=True, check_scale=True)

{39: 39, 40: 40, 41: 41, 58: 58, 59: 59, 60: 60, 63: 63, 64: 64, 65: 65, 76: 76, 77: 77, 78: 78, 79: 79, 80: 80, 81: 81}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 12.619251836766544), ('HarmonicAngleForce', 222.03488526154752), ('PeriodicTorsionForce', 165.54093146134787), ('NonbondedForce', 80580.58719803787), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 12.665182674760329), ('CustomAngleForce', 222.15532081099008), ('CustomTorsionForce', 165.54094120152004), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 12.665182674760329), ('CustomAngleForce', 222.15532081099008), ('CustomTorsionForce', 157.2921325778617), ('AndersenTherm

## Barnase:barstar (old htf, no naked charge fix or counterion fix)

### Test one alchemical region, no scale regions

#### Old system PeroidicTorsionForce vs hybrid system CustomTorsionForce

In [48]:
# Load htf
with open("T42A_complex.pickle", "rb") as f:
    htf = pickle.load(f)

In [49]:
test_torsion_energies(htf, is_solvated=True)

{10866: 2139, 10867: 2140, 10868: 2141, 10907: 2180, 10908: 2181, 10909: 2182, 10912: 2185, 10913: 2186, 10914: 2187, 10922: 2195, 10923: 2196, 10924: 2197, 10925: 2198, 10926: 2199, 10927: 2200}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 190.96991399580065), ('HarmonicAngleForce', 797.0922707362635), ('PeriodicTorsionForce', 3856.2346108037004), ('NonbondedForce', -23496.968598492447), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 191.92874049962995), ('CustomAngleForce', 1012.1096547338736), ('CustomTorsionForce', 3856.2347805821146), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


#### New system HarmonicTorsionForce vs hybrid system CustomTorsionForce

In [50]:
# Load htf
with open("T42A_complex.pickle", "rb") as f:
    htf = pickle.load(f)

In [51]:
test_torsion_energies(htf, is_old=False, is_solvated=True)

{2139: 2139, 2140: 2140, 2141: 2141, 2180: 2180, 2181: 2181, 2182: 2182, 2183: 2183, 2184: 2184, 2185: 2185, 2186: 2186, 2187: 2187, 2188: 2188, 2191: 2191, 2192: 2192, 2193: 2193, 2194: 2194, 2195: 2195, 2196: 2196, 2197: 2197, 2198: 2198, 2199: 2199, 2200: 2200, 2201: 2201, 2202: 2202, 2203: 2203, 2204: 2204, 2205: 2205, 2206: 2206, 2214: 2214, 2215: 2215, 2216: 2216, 2217: 2217, 2218: 2218, 2219: 2219, 2220: 2220, 2221: 2221, 2222: 2222, 2223: 2223, 2224: 2224, 2225: 2225, 2226: 2226, 2227: 2227, 2228: 2228, 2229: 2229, 2230: 2230}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 191.51760393383896), ('HarmonicAngleForce', 1010.4766887798822), ('PeriodicTorsionForce', 3849.6843670339917), ('NonbondedForce', -22772.68423165379), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 191.92874049962995), ('C

### Test one alchemical region, no scale regions

#### Old system HarmonicTorsionForce vs hybrid system CustomTorsionForce

In [7]:
# Load htf
with open("T42A_complex_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [8]:
test_torsion_energies(htf, is_solvated=True, check_scale=True)

{10866: 2139, 10867: 2140, 10868: 2141, 10907: 2180, 10908: 2181, 10909: 2182, 10912: 2185, 10913: 2186, 10914: 2187, 10922: 2195, 10923: 2196, 10924: 2197, 10925: 2198, 10926: 2199, 10927: 2200}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 190.96991399580065), ('HarmonicAngleForce', 797.0922707362635), ('PeriodicTorsionForce', 3856.2346108037004), ('NonbondedForce', -25367.423468547222), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 191.68441661118993), ('CustomAngleForce', 1002.3374923455682), ('CustomTorsionForce', 3856.234842190186), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 191.68441661118993), ('CustomAngleForce', 1002

#### New system HarmonicTorsionForce vs hybrid system CustomTorsionForce

In [9]:
# Load htf
with open("T42A_complex_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [10]:
test_torsion_energies(htf, is_old=False, is_solvated=True, check_scale=True)

{2139: 2139, 2140: 2140, 2141: 2141, 2180: 2180, 2181: 2181, 2182: 2182, 2183: 2183, 2184: 2184, 2185: 2185, 2186: 2186, 2187: 2187, 2188: 2188, 2191: 2191, 2192: 2192, 2193: 2193, 2194: 2194, 2195: 2195, 2196: 2196, 2197: 2197, 2198: 2198, 2199: 2199, 2200: 2200, 2201: 2201, 2202: 2202, 2203: 2203, 2204: 2204, 2205: 2205, 2206: 2206, 2214: 2214, 2215: 2215, 2216: 2216, 2217: 2217, 2218: 2218, 2219: 2219, 2220: 2220, 2221: 2221, 2222: 2222, 2223: 2223, 2224: 2224, 2225: 2225, 2226: 2226, 2227: 2227, 2228: 2228, 2229: 2229, 2230: 2230}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 191.27328004539896), ('HarmonicAngleForce', 1000.7045263915769), ('PeriodicTorsionForce', 3849.683473240853), ('NonbondedForce', -24737.764251905406), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 191.68441661118993), ('C

In [ ]:
# TO DO: actually test the scaling of the energies matches